In [2]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4

cuda


In [3]:
with open('wizard_of_oz.txt', 'r' , encoding='utf-8') as f:
    text = f.read()

print(text[:100])   

  DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ


In [4]:
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


### Tokenizer

In [5]:
string_to_int = {  ch: i for i,ch in enumerate(chars) }
int_to_string = {i:ch for i,ch in enumerate(chars) }
encode = lambda s : [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i]for i in l])

In [6]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


### Splitting the text corpus into train and test split
>Making a block of size 8 and using it to train and predict the target which is the of same block size and the same block but offset by 1 .So basically predicting the next character of the block 

In [7]:
n =  int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size , (batch_size,))
    print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix ] )
    y = torch.stack([data[i+1:i+ block_size + 1] for i in ix])
    x ,y = x.to(device) , y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs : ')
print(x)
print('targets : ')
print(y)

tensor([  3961,  36805, 146822,  30728])


inputs : 
tensor([[72, 72, 62, 72, 73, 54, 67, 56],
        [58, 65, 59,  9,  1, 56, 54, 65],
        [73, 68,  1, 56, 65, 62, 66, 55],
        [57, 22,  1,  3, 31, 62, 57, 10]], device='cuda:0')
targets : 
tensor([[72, 62, 72, 73, 54, 67, 56, 58],
        [65, 59,  9,  1, 56, 54, 65, 65],
        [68,  1, 56, 65, 62, 66, 55,  1],
        [22,  1,  3, 31, 62, 57, 10, 57]], device='cuda:0')


### Making the Bigram Language model

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size , vocab_size)  

    def forward(self, index , targets=None):
        logits =  self.token_embedding_table(index)
        
        if targets is None:
           loss = None
        else:   
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index , max_new_tokens):
        #index is a (B , T) array of indices in the current context
        for i in range(max_new_tokens):
            #get predictions
            logits, loss =  self.forward(index)
            #focus only on the last time step
            logits = logits[: , -1 , :] #Becomes (B, C)
            #apply softmax to get probabilities
            probs =  F.softmax(logits, dim = -1 ) #(B, C)
            #sample form distribution
            index_next = torch.multinomial(probs, num_samples=1) #(B, 1)
            #append samapled index into the running sequence
            index = torch.cat((index, index_next), dim=1) #(B, T+1)

        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype=torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)                


i_81s(c4kO,o)jMGt,.:fEox'kuX'X7YXts6scGp.o&Ssv.!v_VKYXcp-9dUo ;sj?2b﻿jcor[XRmKYJwvAnxzqr51BzYJE siZxC2xdiZUd 2"6JC7TYdS_J:d7Hu0Z1mYXq]2bj0m4Z]﻿k.VZSiUsltrhI 2br]Vw[_PJSqS0- ;aP.F7w[.cJ"ole i2T_eWD9Dw:kl0y(c'VD_es*m0D[MI60Jkm1BTP(vd(lLYXd.o!?S[_B_5YX81KkpqoOTAt3Tk.
ELq1e,W3ziaj*-UzP
kslr47sqr0"3vmd'gVOKqk9D33F0uH51KsIr1xrN4t65K﻿dbj?]y,W:jM7wbc"&[01﻿P9rNWw6Ht3sEzuauBD4J][﻿a(IM4?7P0.or 
8fM;C510]8grSoNA;btuwSZTX

q(vY470:klJp]8tscrF7GAgq:h1ZZ6NTPDVLg[jcjq3eRR381"hIPd?vR﻿xCcMV:dYs0i3HZMIyqH"Q)t3F7LQ


In [9]:
#create a AdamW optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for iter in range(max_iters):

    #sample batch of data
    xb, yb = get_batch('train')

    #evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    print(loss.item())

tensor([ 25225,  85960,  38872, 147445])
4.717346668243408
tensor([ 44973, 116769, 166112,  21280])
4.660436153411865
tensor([ 21951,  63774,  85091, 105625])
4.826685905456543
tensor([123259, 146477,  80286,  53354])
4.572204113006592
tensor([83669,  5682, 79067, 51121])
4.364546775817871
tensor([107640, 156465, 140367, 175977])
4.908842086791992
tensor([ 61946, 141948,  43833, 181540])
4.6440510749816895
tensor([ 35033, 185544,  71020, 124388])
4.826838493347168
tensor([126444, 141852,   6686,   2962])
4.495985984802246
tensor([ 21561, 110796,  84499, 157724])
5.061944007873535
tensor([182191, 140615, 182723,  17749])
4.676394939422607
tensor([151990, 165606,  33790, 142706])
4.873824119567871
tensor([  8224, 137774, 141869,   8773])
4.9126129150390625
tensor([ 99945,  62258,  90988, 162159])
5.010085582733154
tensor([ 43303,  72550, 175079,  44108])
4.882408618927002
tensor([  5297, 166228, 117221,  88180])
4.580659866333008
tensor([141345,  82347,   1111,  41655])
4.929419994354248

In [10]:
context = torch.zeros((1,1),dtype = torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


caulxwE IC1ZDVWB_;jUq:zH3I[Aid;aV﻿4P9wtlepPhemsi3H*Sa
au&AP-E0Dut3x'X7nobpK, tetule st ZAX8re t by
v[!Tliti)LgXt2x" te)BUd-!Kbb6(Z*owas0Q!WZxaSEnd:*w;Khhz.RB﻿outhe t3, nAd"qFL78qgYhyi).fbion'MLqdnme fl0?7ywes6cFand8_OpofYfloUz,'[O(t clzeaedu!PW"iZxzOne:brcHDQkT34Vb4oH*,

cr?C
[stu﻿0WxdnvBS5A6rIcco ,t l,60(haSLxaZK9S
"h7C]]!﻿afle s
w[S"u6U?kAivA"bo9ke tedHgM"O']﻿ Q:Jpeivqu!Z,4?-1e3Y_J]6I2,h
keB*id, bonu6:CVCPOudld'klgim._H[zas(9Y46N06hik1]1-oul m&mO(l!KOf"BMVfMJ1Dhanl. me nd.
s.pGx

"*S"8KfHLKBUF
